Hello, I am JYC. This notebook implements only one sub-task in experimental topic 6. I will explain the logic behind each concept to assist students who are not proficient in coding to quickly grasp the key points and complete the experiment.

In [23]:
import pyglet
import numpy as np

The Boid algorithm simulates the flocking behavior of birds by defining simple rules for each agent (boid) in the system. Each boid has basic attributes such as speed, movement direction, and current position. The movement of an agent is determined by the following equations:

$$ x_i(t+1) = x_i(t) + v_i \cos(\theta_i(t)) \\
 y_i(t+1) = y_i(t) + v_i \sin(\theta_i(t)) $$

Here, $ v_i $ represents the speed, $\theta_i(t)$ is the movement direction, and $(x_i(t), y_i(t))$ is the current position of the boid at time $ t $.

### Basic Rules of the Boid Model

1. **Separation (Avoidance of Collision)**: Boids detect the positions of all other boids within a certain radius. They calculate the centroid of these nearby boids and generate a velocity that moves away from this centroid to avoid collisions.

2. **Alignment (Direction Uniformity)**: Boids detect the velocities of all other boids within a certain radius. They compute the average velocity of these nearby boids and adjust their velocity to align with this average direction, promoting cohesive movement.

3. **Cohesion (Group Aggregation)**: Boids detect the positions of all other boids within a different radius compared to the separation rule. They calculate the centroid of these nearby boids and generate a velocity that moves towards this centroid, helping the boids to stay grouped.


In [24]:
BOIDZ = 150  # How many boids to spawn, too many may slow fps
SPEED = 170  # Movement speed
WIDTH = 1200  # Window Width (1200)
HEIGHT = 800  # Window Height (800)
BGCOLOR = (0, 0, 0)  # Background color in RGB
FPS = 60  # 30-90

In [25]:
class Boid:
    def __init__(self, boid_num, data, batch):
        self.data = data
        self.boid_num = boid_num
        self.boid_size = 17
        self.angle = np.random.randint(0, 360)
        self.pos = np.random.uniform(50, WIDTH - 50, 2)
        self.dir = np.array([1, 0], dtype=np.float64)
        self.triangle = pyglet.shapes.Triangle(
            self.pos[0],
            self.pos[1],
            self.pos[0] - 7,
            self.pos[1] - 11,
            self.pos[0] + 7,
            self.pos[1] - 11,
            color=np.random.randint(0, 256, 3),
            batch=batch,
        )

    def update(self, dt, speed):
        max_w, max_h = WIDTH, HEIGHT
        turn_dir = 0
        turn_rate = 120 * dt
        other_boids = np.delete(self.data, self.boid_num, 0)
        array_dists = (self.pos[0] - other_boids[:, 0]) ** 2 + (self.pos[1] - other_boids[:, 1]) ** 2
        close_boids = np.argsort(array_dists)[:7]
        nbrs = other_boids[close_boids]
        nbrs[:, 3] = np.sqrt(array_dists[close_boids])
        nbrs = nbrs[nbrs[:, 3] < self.boid_size * 12]
        if nbrs.size > 1:
            yat = np.sum(np.sin(np.deg2rad(nbrs[:, 2])))
            xat = np.sum(np.cos(np.deg2rad(nbrs[:, 2])))
            avg_angle = np.rad2deg(np.arctan2(yat, xat))
            target_v = (np.mean(nbrs[:, 0]), np.mean(nbrs[:, 1]))
            if nbrs[0, 3] < self.boid_size:
                target_v = (nbrs[0, 0], nbrs[0, 1])
            diff = np.array(target_v) - self.pos
            distance, angle = np.linalg.norm(diff), np.rad2deg(np.arctan2(diff[1], diff[0]))
            if distance < self.boid_size * 6:
                angle = avg_angle
            angle_diff = (angle - self.angle) + 180
            if abs(angle - self.angle) > 1.2:
                turn_dir = (angle_diff / 360 - (angle_diff // 360)) * 360 - 180
            if distance < self.boid_size and target_v == (nbrs[0, 0], nbrs[0, 1]):
                turn_dir = -turn_dir
        if turn_dir != 0:
            self.angle += turn_rate * abs(turn_dir) / turn_dir
            self.angle %= 360
        self.dir = np.array([np.cos(np.deg2rad(self.angle)), np.sin(np.deg2rad(self.angle))])
        self.pos += self.dir * dt * (speed + (7 - nbrs.size) * 2)
        
        if self.pos[1] < 0:
            self.pos[1] = max_h
        elif self.pos[1] > max_h:
            self.pos[1] = 0
        if self.pos[0] < 0:
            self.pos[0] = max_w
        elif self.pos[0] > max_w:
            self.pos[0] = 0
            
        self.data[self.boid_num, :3] = [self.pos[0], self.pos[1], self.angle]

        self.triangle.x = self.pos[0] + 7 * self.dir[0]
        self.triangle.y = self.pos[1] + 11 * self.dir[1]
        self.triangle.x2 = self.pos[0] - 7 * self.dir[0] + 5 * self.dir[1]
        self.triangle.y2 = self.pos[1] - 11 * self.dir[1] + 5 * self.dir[0]
        self.triangle.x3 = self.pos[0] - 7 * self.dir[0] - 5 * self.dir[1]
        self.triangle.y3 = self.pos[1] - 11 * self.dir[1] - 5 * self.dir[0]


class BoidSimulation(pyglet.window.Window):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.set_location(50, 50)
        self.batch = pyglet.graphics.Batch()
        self.data = np.zeros((BOIDZ, 4), dtype=float)
        self.boids = [Boid(n, self.data, self.batch) for n in range(BOIDZ)]
        pyglet.clock.schedule_interval(self.update, 1 / FPS)

    def on_draw(self):
        self.clear()
        self.batch.draw()

    def update(self, dt):
        for boid in self.boids:
            boid.update(dt, SPEED)

In [26]:
window = BoidSimulation(WIDTH, HEIGHT)
pyglet.app.run()

In [27]:
class Pedestrian:
    def __init__(self, position, destination, speed=1.3, fov=120, dmax=10):
        self.position = np.array(position, dtype=float)
        self.destination = np.array(destination, dtype=float)
        self.speed = speed
        self.fov = np.radians(fov / 2)
        self.dmax = dmax
        self.direction = self.calculate_direction()

    def calculate_direction(self):
        direction = self.destination - self.position
        norm = np.linalg.norm(direction)
        if norm == 0:
            return np.array([0, 0])
        return direction / norm

    def distance_to_collision(self, alpha, obstacles, other_pedestrians):
        min_distance = self.dmax
        for obs in obstacles:
            direction_to_obs = obs - self.position
            distance_to_obs = np.linalg.norm(direction_to_obs)
            angle_to_obs = np.arctan2(direction_to_obs[1], direction_to_obs[0]) - alpha
            if abs(angle_to_obs) <= self.fov:
                min_distance = min(min_distance, distance_to_obs)

        for ped in other_pedestrians:
            direction_to_ped = ped.position - self.position
            distance_to_ped = np.linalg.norm(direction_to_ped)
            angle_to_ped = np.arctan2(direction_to_ped[1], direction_to_ped[0]) - alpha
            if abs(angle_to_ped) <= self.fov:
                min_distance = min(min_distance, distance_to_ped)

        return min_distance

    def calculate_new_direction(self, obstacles, other_pedestrians):
        alpha0 = np.arctan2(
            self.destination[1] - self.position[1],
            self.destination[0] - self.position[0],
        )
        best_direction = alpha0
        min_d = float("inf")
        for alpha in np.linspace(alpha0 - self.fov, alpha0 + self.fov, 100):
            d = (
                self.dmax**2
                + self.distance_to_collision(alpha, obstacles, other_pedestrians) ** 2
                - 2
                * self.dmax
                * self.distance_to_collision(alpha, obstacles, other_pedestrians)
                * np.cos(alpha0 - alpha)
            )
            if d < min_d:
                min_d = d
                best_direction = alpha
        self.direction = np.array([np.cos(best_direction), np.sin(best_direction)])

    def move(self, time_step=0.1, obstacles=[], other_pedestrians=[]):
        self.calculate_new_direction(obstacles, other_pedestrians)
        self.position += self.direction * self.speed * time_step

In [28]:
pedestrians = [
    Pedestrian(position=[2, 2], destination=[10, 10]),
    Pedestrian(position=[10, 0], destination=[0, 10]),
    Pedestrian(position=[5, 5], destination=[10, 0]),
]

obstacles = [np.array([5, 2]), np.array([7, 7])]

time_step = 0.1
num_steps = 100

window = pyglet.window.Window(600, 600)
batch = pyglet.graphics.Batch()


def to_window_coords(x, y):
    return x * 50 + 50, y * 50 + 50


start_points = [
    pyglet.shapes.Circle(
        *to_window_coords(ped.position[0], ped.position[1]),
        5,
        color=(0, 0, 255),
        batch=batch
    )
    for ped in pedestrians
]
end_points = [
    pyglet.shapes.Circle(
        *to_window_coords(ped.destination[0], ped.destination[1]),
        5,
        color=(0, 255, 0),
        batch=batch
    )
    for ped in pedestrians
]
obs_points = [
    pyglet.shapes.Circle(
        *to_window_coords(obs[0], obs[1]), 5, color=(255, 0, 0), batch=batch
    )
    for obs in obstacles
]
paths = [
    pyglet.shapes.Circle(
        *to_window_coords(ped.position[0], ped.position[1]),
        5,
        color=(0, 0, 0),
        batch=batch
    )
    for ped in pedestrians
]


def update(dt):
    for i, ped in enumerate(pedestrians):
        other_peds = [p for j, p in enumerate(pedestrians) if j != i]
        ped.move(time_step, obstacles, other_peds)
        paths[i].x, paths[i].y = to_window_coords(ped.position[0], ped.position[1])


@window.event
def on_draw():
    window.clear()
    batch.draw()

In [29]:
pyglet.clock.schedule_interval(update, time_step)
pyglet.gl.glClearColor(255, 255, 255, 255)
pyglet.app.run()